# <font size="6">Libraries</font>

In [1]:
import pandas as pd
import numpy as np

# <font size="6">dataset</font>

In [2]:
train_df = pd.read_parquet('data/train.parquet').drop(columns=['transaction_number',
                                                               'reversed_transaction', 
                                                               'transaction_max',
                                                               'time'])

valid_df = pd.read_parquet('data/valid.parquet').drop(columns=['transaction_number',
                                                               'reversed_transaction', 
                                                               'transaction_max',
                                                               'time'])

In [3]:
valid_df

,app_id,amnt,mcc,hour_diff,amnt_bins,amnt_mcc_bins,hour_summed,transaction_min
0,0,0.336345,2,0,2,7,0,132
1,0,0.359050,20,74,3,80,74,132
2,0,0.383609,9,0,3,36,74,132
3,0,0.320248,2,23,1,6,97,132
4,0,0.367687,2,0,3,8,97,132
...,...,...,...,...,...,...,...,...
11183192,1003049,0.414108,8,3,3,32,239,60
11183193,1003049,0.251875,4,23,0,13,262,60
11183194,1003049,0.344112,1,1,2,3,263,60
11183195,1003049,0.307540,4,16,1,14,279,60


In [4]:
train_df['mcc'].value_counts()

1      25674831
2      14582198
4       6393060
3       5704688
7       3891901
         ...   
100       32144
103       28461
106       26546
105       21248
91        18953
Name: mcc, Length: 108, dtype: int64

In [7]:
train_df['mcc_next'] = train_df.groupby(['app_id'])['mcc'].shift(-1)
train_df['amnt_next'] = train_df.groupby(['app_id'])['amnt'].shift(-1)
train_df['hour_diff_next'] = train_df.groupby(['app_id'])['hour_diff'].shift(-1)
train_df['amnt_bins_next'] = train_df.groupby(['app_id'])['amnt_bins'].shift(-1)
train_df['amnt_mcc_bins_next'] = train_df.groupby(['app_id'])['amnt_mcc_bins'].shift(-1)

valid_df['mcc_next'] = valid_df.groupby(['app_id'])['mcc'].shift(-1)
valid_df['amnt_next'] = valid_df.groupby(['app_id'])['amnt'].shift(-1)
valid_df['hour_diff_next'] = valid_df.groupby(['app_id'])['hour_diff'].shift(-1)
valid_df['amnt_bins_next'] = valid_df.groupby(['app_id'])['amnt_bins'].shift(-1)
valid_df['amnt_mcc_bins_next'] = valid_df.groupby(['app_id'])['amnt_mcc_bins'].shift(-1)

In [8]:
last_transactions_train = train_df.groupby('app_id').tail(1)

train_df = train_df.drop(last_transactions_train.index)

# train_df = train_df.dropna()

In [14]:
first_transactions_valid = valid_df.groupby('app_id').head(1)
# first_transactions_valid = first_transactions_valid.dropna()

In [31]:
# TODO: уточнить как собиралась валидаций, почему так мало юзеров, у кгог 50 транзакций

valid_df['app_id'].value_counts().value_counts().sort_index()

1      1375
2      2192
3      3215
4      4182
5      5136
6      6034
7      7091
8      8138
9      8680
10     9020
11     9864
12    10178
13    10504
14    10610
15    10876
16    10976
17    10919
18    10995
19    10910
20    10854
21    10784
22    10665
23    10695
24    10567
25    10584
26    10440
27    10341
28    10178
29    10145
30     9845
31     9789
32     9429
33     9413
34     9197
35     9164
36     9192
37     9041
38     8866
39     8512
40     8482
41     8322
42     8024
43     7972
44     8099
45     7820
46     7799
47     7736
48     7993
49     8255
50      613
Name: app_id, dtype: int64

# <font size="6">Model</font>

In [32]:
class MostCommonMCCPredictor:
    def __init__(self):
        self.most_common_mcc = {}

    def fit(self, data):
        self.most_common_mcc = data.groupby('app_id')['amnt_mcc_bins'].apply(lambda x: x.mode().iloc[0]).to_dict()
        
        return self.most_common_mcc

    def forward(self, app_id):
        
        return self.most_common_mcc.get(app_id, None)


model = MostCommonMCCPredictor()

most_common_mcc = model.fit(train_df)

In [41]:
# по юзеру
pd.value_counts(list(most_common_mcc.values()), normalize=True)

8      0.390050
1      0.183817
4      0.182776
13     0.041238
36     0.038984
         ...   
243    0.000002
376    0.000002
304    0.000002
313    0.000002
270    0.000002
Length: 309, dtype: float64

In [43]:
# по датасету
train_df['amnt_mcc_bins'].value_counts(normalize=True).round(6)

8      0.141564
1      0.101093
4      0.097008
3      0.037901
13     0.036598
         ...   
341    0.000008
318    0.000004
317    0.000003
319    0.000002
312    0.000000
Name: amnt_mcc_bins, Length: 430, dtype: float64

In [45]:
last_transactions_train

,app_id,amnt,mcc,hour_diff,amnt_bins,amnt_mcc_bins,mcc_next,amnt_next,hour_diff_next,amnt_bins_next,amnt_mcc_bins_next
130,0,0.348838,2,42,3,8,NaN,NaN,NaN,NaN,NaN
436,1,0.297589,20,4,0,77,NaN,NaN,NaN,NaN,NaN
615,2,0.402386,2,31,3,8,NaN,NaN,NaN,NaN,NaN
682,4,0.621777,9,26,3,36,NaN,NaN,NaN,NaN,NaN
892,6,0.333474,3,3,2,11,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95045500,1003041,0.299564,4,0,0,13,NaN,NaN,NaN,NaN,NaN
95045710,1003044,0.349617,4,0,3,16,NaN,NaN,NaN,NaN,NaN
95045920,1003047,0.229750,18,5,0,69,NaN,NaN,NaN,NaN,NaN
95046089,1003048,0.662458,2,41,3,8,NaN,NaN,NaN,NaN,NaN


In [46]:
%%time
    
predicted_type = []
    
for _, last_transaction in last_transactions_train.iterrows():
    user_id = last_transaction['app_id']
    
    next_transaction = model.forward(last_transaction['app_id'])
    
    predicted_type.append(next_transaction)
    
predicted_type = pd.DataFrame({'predicted_type': predicted_type}).reset_index(drop=True)

CPU times: user 24.7 s, sys: 26.5 ms, total: 24.7 s
Wall time: 24.7 s


In [47]:
predicted_type['app_id'] = last_transactions_train['app_id'].values
predicted_type['pred_mcc'] = (predicted_type['predicted_type'] - 1) // 4 + 1
predicted_type['pred_amnt'] = (predicted_type['predicted_type'] - 1) % 4

predicted_type

,predicted_type,app_id,pred_mcc,pred_amnt
0,8,0,2,3
1,8,1,2,3
2,1,2,1,0
3,36,4,9,3
4,8,6,2,3
...,...,...,...,...
430304,1,1003041,1,0
430305,17,1003044,5,0
430306,16,1003047,4,3
430307,8,1003048,2,3


In [48]:
dataframe = predicted_type.merge(first_transactions_valid, on='app_id', how='inner')

In [49]:
dataframe

,predicted_type,app_id,pred_mcc,pred_amnt,amnt,mcc,hour_diff,amnt_bins,amnt_mcc_bins,hour_summed,transaction_min,mcc_next,amnt_next,hour_diff_next,amnt_bins_next,amnt_mcc_bins_next
0,8,0,2,3,0.336345,2,0,2,7,0,132,20.0,0.359050,74.0,3.0,80.0
1,8,1,2,3,0.439060,2,37,3,8,37,307,1.0,0.323313,0.0,1.0,2.0
2,1,2,1,0,0.320805,28,156,1,110,156,180,2.0,0.485295,10.0,3.0,8.0
3,36,4,9,3,0.387677,9,73,3,36,73,68,9.0,0.393020,0.0,3.0,36.0
4,8,6,2,3,0.411448,58,24,3,232,24,211,8.0,0.384802,1.0,3.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429706,1,1003041,1,0,0.281539,3,17,0,9,17,274,1.0,0.216285,109.0,0.0,1.0
429707,17,1003044,5,0,0.334992,4,0,2,15,0,211,3.0,0.333474,21.0,2.0,11.0
429708,16,1003047,4,3,0.516270,15,42,3,60,42,211,15.0,0.498168,0.0,3.0,60.0
429709,8,1003048,2,3,0.589482,12,143,3,48,143,170,2.0,0.583667,0.0,3.0,8.0


# <font size="6">Metrics</font>

In [51]:
print('Type accuracy:', accuracy_score(dataframe['amnt_mcc_bins'], dataframe['predicted_type']))
print('Amnt accuracy:', accuracy_score(dataframe['amnt_bins'], dataframe['pred_amnt']))
print('MCC accuracy:', accuracy_score(dataframe['mcc'], dataframe['pred_mcc']))

Type accuracy: 0.21812101621787666
Amnt accuracy: 0.5476983367891444
MCC accuracy: 0.3016120136556895


In [57]:
print('Type accuracy:', accuracy_score(
    first_transactions_valid['amnt_mcc_bins'], 
    first_transactions_valid['app_id'].map(most_common_mcc),
))

Type accuracy: 0.21812101621787666


In [62]:
first_transactions_valid['hour_diff'].mean(), first_transactions_valid['hour_diff'].median(), 

(33.73954355369074, 12.0)

In [64]:
first_transactions_valid['hour_diff_next'].mean(), first_transactions_valid['hour_diff_next'].median(), 

(32.96573017444249, 11.0)

In [69]:
valid_df['hour_diff'].mean(), valid_df['hour_diff'].median()

(24.600992900330738, 8.0)

In [78]:
pd.concat([
    valid_df,
    valid_df.assign(rn=1).groupby('app_id')['rn'].cumsum(),
], axis=1).groupby('rn')['hour_diff'].median()

rn
1     12.0
2     11.0
3     11.0
4     11.0
5     11.0
6     11.0
7     11.0
8     10.0
9     10.0
10    10.0
11    10.0
12     9.0
13     9.0
14     9.0
15     9.0
16     8.0
17     8.0
18     8.0
19     8.0
20     8.0
21     7.0
22     7.0
23     7.0
24     7.0
25     7.0
26     7.0
27     7.0
28     7.0
29     7.0
30     6.0
31     6.0
32     6.0
33     6.0
34     6.0
35     6.0
36     6.0
37     6.0
38     6.0
39     6.0
40     6.0
41     6.0
42     6.0
43     6.0
44     6.0
45     6.0
46     6.0
47     7.0
48     7.0
49     8.0
50     5.0
Name: hour_diff, dtype: float64

In [88]:
pd.concat([
    valid_df,
    valid_df.assign(rn=1).groupby('app_id')['rn'].cumsum(),
], axis=1).groupby('rn')['hour_diff'].mean()

rn
1     33.739544
2     32.965730
3     32.223349
4     31.484753
5     30.571476
6     29.890428
7     29.223025
8     28.519916
9     27.714631
10    26.919224
11    26.292031
12    25.669426
13    24.934468
14    24.420695
15    23.823054
16    23.354465
17    22.805843
18    22.359682
19    21.931653
20    21.609036
21    21.133970
22    20.700499
23    20.227384
24    20.021584
25    19.893763
26    19.562518
27    19.176745
28    18.900875
29    18.683117
30    18.437634
31    18.108748
32    17.839848
33    17.749363
34    17.565458
35    17.727945
36    17.503812
37    17.324679
38    17.314398
39    17.226372
40    17.437342
41    17.017224
42    17.269394
43    17.385666
44    17.789258
45    18.298438
46    18.519632
47    19.712485
48    20.373050
49    23.324650
50    15.208809
Name: hour_diff, dtype: float64

In [93]:
50 * 15 / 24, 30 * 18 / 24

(31.25, 22.5)

In [94]:
train_df['hour_diff'].median()

7.0

In [97]:
train_df.groupby('app_id')['hour_diff'].median().median()

11.5

In [81]:
valid_df.groupby('app_id').tail(1)['hour_diff'].median()

17.0

In [83]:
valid_df.groupby('app_id').tail(2)['hour_diff'].median()

16.0

In [103]:
abs(
    valid_df.groupby('app_id').head()['hour_diff'] -
    train_df['hour_diff'].median(),
).mean()

30.29193124499322

In [104]:
abs(
    valid_df.groupby('app_id').head()['hour_diff'] -
    train_df.groupby('app_id')['hour_diff'].median().median(),
).mean()

30.12039017492628

In [105]:
abs(
    valid_df['hour_diff'] -
    train_df['hour_diff'].median(),
).mean()

22.926650670644538

In [106]:
abs(
    valid_df['hour_diff'] -
    train_df.groupby('app_id')['hour_diff'].median().median(),
).mean()

23.00553066354818